# 단위 테스트 도입

## 단일 모듈 테스트

In [1]:
!python3 -m unittest booksearch_module.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [2]:
# -v 옵션으로 상세 정보 표시
!python3 -m unittest -v booksearch_module.py

test_booksearch (booksearch_module.BookSearchTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### 테스트 실행 명령어 간략화

In [1]:
!cat booksearch_module.py

import unittest


# 애플리케이션 코드
def booksearch():
    # 임의 처리
    return {}


class BookSearchTest(unittest.TestCase):
    # booksearch() 테스트 코드
    def test_booksearch(self):
        self.assertEqual({}, booksearch())


if __name__ == '__main__':
    unittest.main()


In [4]:
!python3 booksearch_module.py -v

test_booksearch (__main__.BookSearchTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## 패키지 테스트

### 디렉터리 구성

In [2]:
# booksearch/, tests/를 준비한 디렉터리로 이동합니다
%cd workspace

/Users/yeonsookim/Workspaces/personal/python-src/12-unittest/workspace


### 샘플 애플리케이션 작성

In [5]:
!cat booksearch/__init__.py

from .core import Book, get_books
__all__ = ['Book', 'get_books']

In [3]:
!cat booksearch/api.py

import json
from urllib import request, parse

def get_json(param):
    with request.urlopen(build_url(param)) as f:
        return json.load(f)

def get_data(url):
    with request.urlopen(url) as f:
        return f.read()

def build_url(param):
    query = parse.urlencode(param)
    return ('https://www.googleapis.com'
            f'/books/v1/volumes?{query}')

In [4]:
!cat booksearch/core.py

import imghdr
import pathlib
from .api import get_data, get_json


class Book:
    """API 응답의 VolumeInfo 엘리먼트에 대응"""

    def __init__(self, item):
        self.id = item['id']
        volume_info = item['volumeInfo']
        for k, v in volume_info.items():
            setattr(self, str(k), v)

    def __repr__(self):
        return str(self.__dict__)

    def save_thumbnails(self, prefix):
        """섬네일 이미지를 저장함"""
        paths = []
        for kind, url in self.imageLinks.items():
            thumbnail = get_data(url)
            # 이미지 데이터로부터 확장자 판정
            ext = imghdr.what(None, h=thumbnail)
            # pathlib.Path는 / 연산자로 경로를 추가할 수 있음
            base = pathlib.Path(
                prefix) / f'{self.id}_{kind}'
            filename = base.with_suffix(f'.{ext}')
            filename.write_bytes(thumbnail)
            paths.append(filename)
        return paths


def get_books(q, **params):
    """서적 검색 수행"""
    params['q'] = q
    data = get_json(params)
    return [Boo

In [6]:
from booksearch import get_books
books = get_books(q='python')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1108)>

In [7]:
# 실행 시 얻은 데이터에 따라 결과가 달라짐
books[0]

NameError: name 'books' is not defined

# unittest 모듈 ── 표준 단위 테스트 라이브러리

## 테스트 케이스 구현

### 전처리, 후처리가 필요한 테스트 케이스

## 테스트 실행과 결과 확인

### 테스트 실패 시 결과

### 테스트 실패 시 결과 억제하기

## 특정 테스트만 실행하기

### 테스트 케이스 직접 실행

### 테스트 디스커버리

# unittest.mock 모듈 ── 목(모의) 객체 이용

## 목 객체 기본 사용법

### 임의의 값을 반환하는 호출 가능 객체로서 이용

In [8]:
from unittest.mock import Mock

# 인수에 반환값을 설정
mock = Mock(return_value=3)
mock()

3

In [9]:
# return_value는 나중에도 설정할 수 있음
mock.return_value=4

# 호출 시 인수는 반환값에 영향 없음
mock(1)

4

In [10]:
# 함수에서는 인수가 그대로 전달됨
mock = Mock(side_effect=lambda x: x % 2)
mock(3)

1

In [11]:
# side_effect는 나중에도 설정할 수 있음
# 이터러블을 사용하면 호출할 때마다 앞에서부터 순서대로 반환됨
mock.side_effect=[2, 1]
mock()

2

In [12]:
mock()

1

In [14]:
# 예외 클래스나 그 인스턴스에서는 해당 예외가 전송됨
mock.side_effect = ValueError('에러입니다')
mock()

ValueError: 에러입니다

### Assert 메서드로 호출 여부 테스트

In [15]:
mock = Mock(return_value=3)

# 아직 한 번도 호출되지 않았음을 확인
mock.assert_not_called()

In [17]:
# 한 번만 호출되었음을 확인
# 아직 한 번도 호출되지 않았으므로 에러가 발생함
mock.assert_called_once()

AssertionError: Expected 'mock' to have been called once. Called 0 times.

In [18]:
# 호출해 봄
mock(1, a=2)

3

In [24]:
# 호출되었으므로 에러 발생
mock.assert_not_called()

AssertionError: Expected 'mock' to not have been called. Called 1 times.
Calls: [call(1, a=2)].

In [25]:
# 한 번만 호출된 것을 확인
mock.assert_called_once()

In [26]:
# 호출 여부를 확인
mock.assert_called_once_with(1, a=2)

In [27]:
mock.assert_called_once_with(1, a=3)

AssertionError: expected call not found.
Expected: mock(1, a=3)
Actual: mock(1, a=2)

In [23]:
# 호출된 횟수는 확인하지 못하며, 일부 인수만 확인
from unittest.mock import ANY
mock.assert_called_with(1, a=ANY)

## patch를 사용한 객체 치환

In [29]:
from booksearch import get_books
from unittest.mock import patch

# 대화형 모드에서는 __main__ 모듈에서 이름을 지정
with patch('__main__.get_books') as mock_get_books:
    mock_get_books.return_value = []
    print(get_books())

[]


In [30]:
@patch('__main__.get_books')
def test_use_mock(mock_get_books):
    mock_get_books.return_value = []
    return get_books()

In [31]:
test_use_mock()

[]

## mock을 이용한 테스트 케이스 실제 사례

In [28]:
from booksearch import get_books
book = get_books(q='python')[0]

In [29]:
# 실행 시에 얻은 데이터에 따라 결과는 다름
book.save_thumbnails('tests/data')

[PosixPath('tests/data/oW63DwAAQBAJ_smallThumbnail.jpeg'),
 PosixPath('tests/data/oW63DwAAQBAJ_thumbnail.jpeg')]

# 유스 케이스 별 테스트 케이스 구현

## 환경 의존 테스트 건너 뛰기

## 예외 발생 테스트

In [30]:
!python3 -m unittest tests.test_core.GetBooksTest -v

test_get_books_no_connection (tests.test_core.GetBooksTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.086s

OK


## 다른 파라미터로 동일한 테스트 반복하기

In [31]:
!python3 -m unittest tests.test_api.BuildUrlMultiTest


FAIL: test_build_url_multi (tests.test_api.BuildUrlMultiTest) (q='python', maxResults=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/suyamar/github/python-practice-book/src/12-unittest/workspace/tests/test_api.py", line 56, in test_build_url_multi
    self.assertEqual(expected, actual)
AssertionError: 'https://www.googleapis.com/books/v1/volumes?q=python' != 'https://www.googleapis.com/books/v1/volumes?q=python&maxResults=1'
- https://www.googleapis.com/books/v1/volumes?q=python
+ https://www.googleapis.com/books/v1/volumes?q=python&maxResults=1
?                                                     +++++++++++++


FAIL: test_build_url_multi (tests.test_api.BuildUrlMultiTest) (q='python', langRestrict='en')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/suyamar/github/python-practice-book/src/12-unittest/workspace/tests/test_api.py", lin

## 컨텍스트 관리자 테스트하기

In [32]:
!python3 -m unittest tests.test_api.GetJsonTest -v

test_get_json (tests.test_api.GetJsonTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.057s

OK


# 정리